In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

### Obtain and preprocess data

In [ ]:
df=pd.read_csv("../input/trip-advisor-hotel-reviews/tripadvisor_hotel_reviews.csv")
df.head()

In [ ]:
df["Rating"].value_counts()

In [ ]:
# If rating less than 3 then is negative(1) else positive(0)
def getSentiment(rating):
    if rating<=3:
        sentiment=0
    else:
        sentiment=1   
    return sentiment

In [ ]:
df["Sentiment"]=df["Rating"].map(getSentiment)
df.drop(columns=["Rating"],inplace=True)    # remove rating column since not needed
df.head()

### Split data into training and validating

In [ ]:
from sklearn.model_selection import train_test_split

X=df["Review"]
y=df["Sentiment"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
print(X_train.shape,X_test.shape)

In [ ]:
max_len=len(df["Review"][0].split(" "))
print(max_len)

for i in range(1,20491):
    cur_len=len(df["Review"][i].split(" "))
    #print(cur_len)
    if max_len<cur_len:
        max_len=cur_len
    else:
        continue
print(max_len)

### Tokenize the words 

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
#use 5000 most commmon words

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)



### Pad the sequences

In [ ]:
from keras.preprocessing import sequence
max_len=250

# if less than 250 will add 0 at the front if less, will remove from the end if more

X_train=sequence.pad_sequences(X_train,maxlen=max_len,truncating="post")
X_test=sequence.pad_sequences(X_test,maxlen=max_len,truncating="post")

In [ ]:
VOCAB_SIZE=len(tokenizer.word_index)+1
VOCAB_SIZE

### Model creation

In [ ]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.summary()

In [ ]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=["acc"])

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5)

In [ ]:
count=0
for i,j in tokenizer.word_index.items():
    print("Key:",i)
    print("Value:",j)
    print("break")
    count+=1
    if count>=5:
        break


In [ ]:
from tensorflow import keras
# sample_text = 'This is a sample sentence.'
# tf.keras.preprocessing.text.text_to_word_sequence(sample_text)
# ["this","is","a","sample","sentence"]

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  # find index of word if exists else make it 0
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens],max_len)
  #return sequence.pad_sequences([tokens],max_len)[0]=access only the 1D array

In [ ]:
encode_text("This hotel is great")

### Make predictions

In [ ]:
import numpy as np

def predict(text):
    encoded_text = encode_text(text)
    #pred = np.zeros((1,250))
    #pred[0] = encoded_text
    #result = model.predict(pred) 
    result=model.predict(encoded_text)
    if result[0]<=0.5:
        sentiment="Negative"
    else:
        sentiment="Positive"
    return sentiment

In [ ]:
text="Best hotel ever.The room are very nice and spacious.The staffs are very friendly and kind."
predict(text)



In [ ]:
text2="This is the worst hotel that I have every stayed in.The rooms are very dirty and small."
predict(text2)

In [ ]:
model.save("my_model")